In [ ]:
import os
import copy
import torch
import torchvision
import torchprune as tp
import torch.nn as nn
import argparse
import nn_utils
class arguments():
    def __init__(self):
        self.batch_size=64
        self.test_batch_size=1000
        self.lr=.2
        self.epochs=200
        self.log_interval=10
        self.verbose=False
        self.prune_batch_size=1000
        self.dataset="cifar10"
        self.use_valid=True
        self.ft_proportion=0
        self.fine_tune=False
        self.arch="VGG16"
        self.load_fname="vgg16_epo160_seed2_best.pt"
args=arguments()

from nn_utils import train, test, load_model
model_full=load_model(args)
model=model_full
def test(args, model, device, test_loader, criterion, epoch, returnAcc=False, pstatement="Test"):
    model.eval()
    test_loss = 0
    correlation=0
    correct_1, correct_5, total = 0, 0, 0

    with torch.no_grad():
        for data, target in test_loader:

            data  = data.to(device, non_blocking=True)
            target = target.to(device, non_blocking=True)
            output = model(data)
            with torch.no_grad():
                original=model_full(data)
                labels=original.argmax(dim=1, keepdim=True)
            
            if args.dataset == "fashion-mnist" \
                    or args.dataset == "cifar10":
                test_loss += criterion(output, target).item()  
                pred = output.argmax(dim=1, keepdim=True)  

                correlation+=(np.sum(pred.numpy()==labels.numpy()))

                correct_1 += pred.eq(target.view_as(pred)).sum().item()
            total += len(target)
    test_loss /= len(test_loader) # over num batches

    if epoch % args.log_interval == 0 and args.verbose:
        if args.dataset == "patches":
            print('{:s}: \t\t\tAvg Loss: {:f}\n'.format(pstatement, test_loss), flush=True)
        elif args.dataset == "fashion-mnist" or args. dataset == "cifar10":
            print('{:s}: Avg loss: {:f}, Accuracy: {}/{} ({:.1f}%) Top5 Accuracy: {}/{} ({:.1f}%)'.\
                    format(pstatement, 
                    test_loss, correct_1, total,
                    100. * correct_1 / total, 
                    correct_5, total,
                    100. * correct_5 / total),
                    flush=True)
    if returnAcc:
        return test_loss, correct_1 / total, correct_5 / total, correlation/total
    return test_loss





from data_utils import fashionmnist_loader, cifar10_loader
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

args.k_args="fracSkipIter"
args.skip=[]
trial=4
torch.manual_seed(trial)
np.random.seed(trial)
train_loader, test_loader, prune_loader=cifar10_loader(args)
import pickle
file=open("prundModel.p", 'rb')
model=pickle.load(file)
file.close()

In [ ]:
batch_size = 32
train_params = {
    # any loss and corresponding kwargs for __init__ from tp.util.nn_loss
    "loss": "CrossEntropyLoss",
    "lossKwargs": {"reduction": "mean"},
    # exactly two metrics with __init__ kwargs from tp.util.metrics
    "metricsTest": [
        {"type": "TopK", "kwargs": {"topk": 1}},
        {"type": "TopK", "kwargs": {"topk": 5}},
    ],
    # any optimizer from torch.optim with corresponding __init__ kwargs
    "optimizer": "SGD",
    "optimizerKwargs": {
        "lr": 0.1,
        "weight_decay": 1.0e-4,
        "nesterov": False,
        "momentum": 0.9,
    },
    # batch size
    "batchSize": batch_size,
    # desired number of epochs
    "startEpoch": 0,
    "retrainStartEpoch": -1,
    "numEpochs": 5,  # 182
    # any desired combination of lr schedulers from tp.util.lr_scheduler
    "lrSchedulers": [
        {
            "type": "MultiStepLR",
            "stepKwargs": {"milestones": [91, 136]},
            "kwargs": {"gamma": 0.1},
        },
        {"type": "WarmupLR", "stepKwargs": {"warmup_epoch": 5}, "kwargs": {}},
    ],
    # output size of the network
    "outputSize": 10,
    # directory to store checkpoints
    "dir": os.path.realpath("./checkpoints"),
}
retrain_params = copy.deepcopy(train_params)
trainer = tp.util.train.NetTrainer(
    train_params=train_params,
    retrain_params=retrain_params,
    train_loader=train_loader,
    test_loader=test_loader,
    valid_loader=prune_loader,
    num_gpus=0,
)
loss_handle = trainer.get_loss_handle()
net_name = "vg16_CIFAR10"
model.eval()
net = tp.util.net.NetHandle(model, net_name)
from nn_utils import print_model_param_flops
sizes=[.6, .5, .45, .4, .35, .3,.25, .2,.15, .1]
params=[]
accs=[]
flops=[]
for keep_ratio in sizes:
    print(keep_ratio)
    
    net_weight_pruned = tp.LearnedRankNet(net, prune_loader, loss_handle)
    
    test(args, net_weight_pruned, 'cpu', prune_loader, criterion = nn.CrossEntropyLoss(), epoch=160, returnAcc=True)
    net_weight_pruned.compress(keep_ratio=keep_ratio)
    flops.append(print_model_param_flops(net_weight_pruned.compressed_net.torchnet, input_res=32)[0])
    params.append(net_weight_pruned.size())
    accs.append(test(args, net_weight_pruned, 'cpu', test_loader, criterion = nn.CrossEntropyLoss(), epoch=160, returnAcc=True))
    print(accs)
    
accs=np.array(accs)
params=np.array(params)
file=open('LRank+IDNet.p', 'wb')
import pickle

pickle.dump([params, accs, flops],file)
file.close()

In [ ]:
from nn_utils import print_model_param_flops
import importlib
importlib.reload(nn_utils)
from nn_utils import print_model_param_flops
#print(net_weight_pruned.compressed_net.torchnet)
print(print_model_param_flops(net_weight_pruned.compressed_net.torchnet, input_res=32)[0])

In [ ]:
print_model_param_flops(model_full, input_res=32)[0]

In [ ]:
31282/62788

In [ ]:
test(args, model, 'cpu', test_loader, criterion = nn.CrossEntropyLoss(), epoch=160, returnAcc=True)

In [ ]:
model_full=load_model(args)
test(args, model_full, 'cpu', test_loader, criterion = nn.CrossEntropyLoss(), epoch=160, returnAcc=True)